In [1]:
import numpy as np
import pandas as pd
import pyesg
from arrays import invest_array, salary_array, age_array
from allocation import allocation_array
from mortality import mortality_sim

periods_per_year = 2
years = 60

age = 30
mi = .99
sex = 'M'
smoker = 'NS'
ret_age = 65
fund_value_start = 100000
withdraw_amount = 1000000
salary = 100000
salary_growth = .045
invest_pct = .15
inflation = .02
stock_pct_strt = .80
bond_pct_strt = 1 - stock_pct_strt
five_yr_pct = .55
stock_pct_end = .2
bond_pct_end = 1 - stock_pct_end

# instantiate a new model with the required parameters
stock_model = pyesg.GeometricBrownianMotion(mu=0.10, sigma=0.15)
bond_model = pyesg.GeometricBrownianMotion(mu=0.06, sigma=0.05)

x0 = 100.0                             # the start value of our process
dt = 1/periods_per_year                # the length of each timestep in years
n_scenarios = 10                       # the number of scenarios to generate
n_steps = periods_per_year * years     # the number of time steps per scenario
# random_state = 259                   # optional random_state for reproducibility

s_model_results = stock_model.scenarios(x0, dt, n_scenarios, n_steps)
b_model_results = bond_model.scenarios(x0, dt, n_scenarios, n_steps)

# create stock and bond index return arrays. 
stock_return = s_model_results[:, 1:] / s_model_results[:, :-1]
bond_return = b_model_results[:, 1:] / b_model_results[:, :-1]

stock_array = np.insert(stock_return, 0, fund_value_start * stock_pct_strt, axis=1)
bond_array = np.insert(bond_return, 0, fund_value_start * bond_pct_strt, axis=1)

# the last return value is not used so we add a 1 to the end to return the array to its original length
ones_to_append = np.ones((stock_return.shape[0], 1), dtype=int)
stock_return = np.append(stock_return, ones_to_append, axis=1)
bond_return = np.append(bond_return, ones_to_append, axis=1)

salary_arr = salary_array(salary = salary, salary_growth= salary_growth, periods_per_year=periods_per_year, years=years, age = age, ret_age=ret_age)
invest_arr = invest_array(salary_array = salary_arr, invest_pct = invest_pct, periods_per_year = periods_per_year, withdraw_amount=withdraw_amount, age = age, ret_age=ret_age)

mortality_arr = mortality_sim(age = age, ret_age = ret_age, sex = sex, smoker = smoker, n_scenarios = n_scenarios, mi = mi)
allocation_arr = allocation_array(
    start_pct = stock_pct_strt,        
    five_yr_pct = five_yr_pct,
    end_pct = stock_pct_end,
    age = age,             
    ret_age = ret_age,    
    periods_per_year = periods_per_year,   
    years = years 
)            


for s, b, inv, alloc_s, alloc_b in zip(stock_array, bond_array, invest_arr, allocation_arr[0], allocation_arr[1]):
    for i in range(1, len(s)):
        s[i] = s[i-1] * s[i]
        b[i] = b[i-1] * b[i]
        total_fund = s[i] + b[i] + invest_arr[i-1]/periods_per_year
        s[i] = total_fund * alloc_s
        b[i] = total_fund * alloc_b

total_fund = stock_array + bond_array
age_arr = age_array(age = age, periods_per_year=periods_per_year, years=years)
age_arr = age_arr.reshape(1, -1)

invest_arr = invest_arr.reshape(1, -1)

age_names = ['age_' + str(i+1) for i in range(len(age_arr))]
stock_fund_names = ['stock_fund_value_' + str(i+1) for i in range(len(stock_array))]
stock_return_names = ['stock_return_' + str(i+1) for i in range(len(stock_array))]
bond_fund_names = ['bond_fund_value_' + str(i+1) for i in range(len(bond_array))]
bond_return_names = ['bond_return_' + str(i+1) for i in range(len(bond_array))]
invest_array_names = ['invest_array_' + str(i+1) for i in range(len(invest_arr))]

total_fund_df = pd.DataFrame(total_fund)
age_array_df = pd.DataFrame(age_arr, index=age_names)
invest_array_df = pd.DataFrame(invest_arr, index=invest_array_names)
stock_array_df = pd.DataFrame(stock_array, index=stock_fund_names)
stock_return_df = pd.DataFrame(stock_return, index=stock_return_names)
bond_array_df = pd.DataFrame(bond_array, index=bond_fund_names)
bond_return_df = pd.DataFrame(bond_return, index=bond_return_names)

df = pd.concat([age_array_df, stock_array_df, stock_return_df, bond_array_df, bond_return_df, invest_array_df], axis=0)
df2 = total_fund_df.T
arrays_dict = {
    'stock_array': stock_array,
    'stock_return': stock_return,
    'bond_array': bond_array,
    'bond_return': bond_return,
    'invest_arr': invest_arr,
    'allocation_arr': allocation_arr,
    'age_arr': age_arr
}

for name, arr in arrays_dict.items():
    print(f'{name}: {np.shape(arr)}')

with pd.ExcelWriter('Outputs/retirement_calc.xlsx') as writer:
    df.to_excel(writer, sheet_name='sheet1')

stock_array: (10, 121)
stock_return: (10, 121)
bond_array: (10, 121)
bond_return: (10, 121)
invest_arr: (1, 121)
allocation_arr: (2, 121)
age_arr: (1, 121)


In [2]:
for col in df2.columns:
    idx = (df2[col] < 0).idxmax()
    if df2[col][idx] < 0:  # Check to ensure the value at the index is indeed negative
        print(f"Column '{col}' first negative value at index {idx}")
    else:
        print(f"Column '{col}' has no negative values")




Column '0' has no negative values
Column '1' first negative value at index 81
Column '2' first negative value at index 112
Column '3' first negative value at index 83
Column '4' has no negative values
Column '5' first negative value at index 102
Column '6' first negative value at index 84
Column '7' first negative value at index 82
Column '8' first negative value at index 89
Column '9' has no negative values
